In [64]:
from game import GlobalFeatureImportance, LocalFeatureImportance, UnsupervisedFeatureImportance
from environment import FixedBudgetEnvironment
from algorithms.SVARM import StratSVARM
from algorithms.CMCS import CMCS, CMCS_Dependent, CMCS_Independent, CMCS_Length, Selective_CMCS
from algorithms.KernelSHAP import KernelSHAP
from algorithms.ApproShapley import ApproShapley
import numpy as np
import pandas as pd
import os
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
n=14
K=np.arange(n-1)+1
budget=1500
rounds=1000
step_interval=50
metric = "ratio"
env = FixedBudgetEnvironment(n=n, budget=budget)

In [66]:
filepath_global = "datasets/Global feature importance/Adult classification random forest.csv"
filepath_unsupervised = "datasets/Unsupervised Feature Importance/vf_Fifa.csv"
directory_local = "datasets/image classifier"
game = GlobalFeatureImportance(filepath=filepath_global, num_players=n, use_cached=True)
# game = LocalFeatureImportance(directory=directory_local, num_players=n)
# game = UnsupervisedFeatureImportance(filepath=filepath_unsupervised, num_players=n)
# game = SumUnanimityGames()

[0.         0.         0.00276384 ... 0.09182107 0.08168697 0.08967141]
[ 0.00698709  0.00212931 -0.02469719  0.00913919  0.0099256   0.01166912
  0.00945903  0.01234841 -0.00015445  0.00210819  0.03377652  0.01190902
  0.00521593 -0.00014437] 0.0896714095608559


In [67]:
algorithms = [("KernelSHAP", KernelSHAP()), ("StratSVARM(False,False)", StratSVARM(start_exact=False, theoretical_distribution=False))]#

In [68]:
algorithms += [("ApproShapley(optimized)", ApproShapley(optimize=True)), ("Selective_CMCS", Selective_CMCS()), ("CMCS", CMCS())]

In [69]:
results = []
columns = ["K", "topk_ratio", "topk_ratio_SE"]
for name, algorithm in algorithms:
    print(name)
    result = env.evaluate(game, algorithm, K=K, rounds=rounds)
    results += [(name, result)]
    df = pd.DataFrame(data={column: result[index] for index, column in enumerate(columns)})
    dir = f"results/data/{game.__class__.__name__}/{game.name}/budget={budget}_rounds={rounds}/k={K[0]}-{K[-1]}"
    if not os.path.isdir(dir):
        os.makedirs(dir)
    df.to_csv(f"{dir}/{name}.csv", index=False)

KernelSHAP
StratSVARM(False,False)
ApproShapley(optimized)
Selective_CMCS
CMCS
